the original notebook is from hugging face colab notebook here (https://colab.research.google.com/github/sanchit-gandhi/notebooks/blob/main/fine_tune_whisper.ipynb)


Make sure you have the following dependencies installed in your environment

```
pip install datasets transformers evaulate jiwer
```

the common voice dataset is coming from here (https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0/viewer/zh-HK/train?p=84)

It is part of mozilla foundation common voice project

----

We could use this format to prepare our own dataset to fine tune our version of whisper

----

If you want to re-use / avoid to download the voice file every time, you can un-comment the part which specify `cache_dir` and point it to the directory you want those file to be downloaded / already downloaded.

In [5]:
# import sys
# sys.path.append(datasets_dir)

from datasets import load_dataset, DatasetDict

dataset_name = "mozilla-foundation/common_voice_11_0"
language_to_train = 'yue'

common_voice = DatasetDict()
common_voice["train"] = load_dataset(
  dataset_name, language_to_train, 
  split="train+validation",
  # cache_dir="/Volumes/BACKUP/Coding/HUGGING_FACE/datasets"
  )

common_voice["test"] = load_dataset(
  dataset_name, language_to_train, 
  split="test",  
  # cache_dir="/Volumes/BACKUP/Coding/HUGGING_FACE/datasets"
  )

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 5296
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2438
    })
})


In [6]:
# !pip install "tokenizers>=0.14,<0.15"

from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained(
  "openai/whisper-small", 
  cache_dir="/Volumes/BACKUP/Coding/HUGGING_FACE/models"
  ) # start with the whisper small checkout

In [9]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", 
language="cantonese", 
task="transcribe"
)

tokenizer_config.json: 100%|██████████| 805/805 [00:00<00:00, 765kB/s]
vocab.json: 100%|██████████| 836k/836k [00:00<00:00, 3.05MB/s]
tokenizer.json: 100%|██████████| 2.48M/2.48M [00:00<00:00, 5.95MB/s]
merges.txt: 100%|██████████| 494k/494k [00:00<00:00, 35.1MB/s]
normalizer.json: 100%|██████████| 52.7k/52.7k [00:00<00:00, 42.3MB/s]
added_tokens.json: 100%|██████████| 34.6k/34.6k [00:00<00:00, 29.3MB/s]
special_tokens_map.json: 100%|██████████| 2.08k/2.08k [00:00<00:00, 5.48MB/s]


In [10]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", 
language="cantonese", 
task="transcribe"
)

preprocessor_config.json: 100%|██████████| 185k/185k [00:00<00:00, 1.90MB/s]


In [11]:
# Preparing Data

print(common_voice["train"][0])

# Whisper expecting the audio to be at sampling rate @16000 - this is just to make sure the sampling rate fits whisper's training
# Since our input audio is sampled at 48kHz, we need to downsample it to 16kHz prior to passing it to the Whisper feature extractor, 
# 16kHz being the sampling rate expected by the Whisper model.
from datasets import Audio
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print(common_voice["train"][0])

{'client_id': 'b472f8e5800db5f400e4b5858c16d68d205f19888992c65770b5a3852c5bfc0cb94a22890f49ca720dff40b3a1e8d466c2290a3480b916df03ea07322657f0d8', 'path': '/Volumes/BACKUP/Coding/HUGGING_FACE/datasets/downloads/extracted/512cb97225d04177325e321598281e08f0112d7fbb640b4e8d8f72d4962fae2e/yue_train_0/common_voice_yue_31209989.mp3', 'audio': {'path': '/Volumes/BACKUP/Coding/HUGGING_FACE/datasets/downloads/extracted/512cb97225d04177325e321598281e08f0112d7fbb640b4e8d8f72d4962fae2e/yue_train_0/common_voice_yue_31209989.mp3', 'array': array([ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
       -7.54985740e-06, -5.66183462e-06, -1.75592550e-06]), 'sampling_rate': 48000}, 'sentence': '美國都係唔考慮喇，睇下澳洲先', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'yue', 'segment': ''}
{'client_id': 'b472f8e5800db5f400e4b5858c16d68d205f19888992c65770b5a3852c5bfc0cb94a22890f49ca720dff40b3a1e8d466c2290a3480b916df03ea07322657f0d8', 'path': '/Volumes/BACKUP/Coding/HUGGING_FA

prepare the dataset

In [12]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch



common_voice = common_voice.map(prepare_dataset, 
  remove_columns=common_voice.column_names["train"], 
  num_proc=1)
print(common_voice)

Map: 100%|██████████| 2438/2438 [01:06<00:00, 36.50 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5296
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2438
    })
})


the following is the actual training and evaluation of the model

using the trainer provided by huggingface

Evaluation metrics: during evaluation, we want to evaluate the model using the word error rate (WER) metric. We need to define a compute_metrics function that handles this computation.

Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

In [13]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Evaluation using hugging face metric - WER (Word error rate)

In [16]:
!pip install evaluate

  Using cached evaluate-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)


In [17]:
import evaluate

metric = evaluate.load("wer")


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [18]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(
  "openai/whisper-small", 
  # cache_dir="/Volumes/BACKUP/Coding/HUGGING_FACE/models"
  )

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

config.json: 100%|██████████| 1.97k/1.97k [00:00<00:00, 1.08MB/s]
model.safetensors: 100%|██████████| 967M/967M [00:09<00:00, 106MB/s]  
generation_config.json: 100%|██████████| 3.84k/3.84k [00:00<00:00, 1.05MB/s]


What should be the training

In [ ]:
!pip install tensorboardx

In [24]:
from transformers import Seq2SeqTrainingArguments
import datetime

now = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M")

training_args = Seq2SeqTrainingArguments(
    output_dir="model/whisper-small-cantanese_"+now,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=False,  # if we are not using CUDA or non graphics card, use fp16=false
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"], #this would requires the tensorboardx to be installed
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    checkpoint_activations=True
)

processor.save_pretrained(training_args.output_dir)

The actual Training Part

In [26]:
trainer.train()

  1%|          | 25/4000 [02:33<6:48:55,  6.17s/it]

{'loss': 1.7712, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.08}


  1%|▏         | 50/4000 [05:09<6:54:01,  6.29s/it]

{'loss': 1.3822, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.15}


  2%|▏         | 75/4000 [07:46<6:51:39,  6.29s/it]

...

{'loss': 0.0004, 'learning_rate': 0.0, 'epoch': 12.08}


                                                     
100%|██████████| 4000/4000 [7:54:19<00:00,  6.13s/it]

{'eval_loss': 0.19237440824508667, 'eval_wer': 64.4561261625556, 'eval_runtime': 854.6178, 'eval_samples_per_second': 2.853, 'eval_steps_per_second': 0.357, 'epoch': 12.08}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
100%|██████████| 4000/4000 [7:54:30<00:00,  7.12s/it]

{'train_runtime': 28470.8418, 'train_samples_per_second': 2.248, 'train_steps_per_second': 0.14, 'train_loss': 0.04882594305602834, 'epoch': 12.08}


if you need to push the model to hugging face hub, run the following block

```
pip install --upgrade huggingface_hub
```

In [ ]:
# this is optional. but it would allow you to upload the model to hugging face space later on
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# to push

#the following arguments are needed only when we are pushing the model to hugging face hub
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "Cantonese",
    "model_name": "[language-x-change] Custom Whisper for Cantanese",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

trainer.push_to_hub(**kwargs)

The following is only needed when we want to deploy a runnable version with our uploaded model on hugging face spaces

In [ ]:
!pip install gradio

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="your-own-model")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

to use the model we just compiled (https://huggingface.co/docs/transformers/tasks/asr#inference)


In [9]:
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor
import datetime
import json

path = "model/whisper-small-cantanese/checkpoint-4000"
processor_path = "model/whisper-small-cantanese"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
   path, 
   local_files_only=True,
)

processor = AutoProcessor.from_pretrained(processor_path)

transcriber = pipeline("automatic-speech-recognition", 
    model=model,  
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    # chunk_length_s=10,
    max_new_tokens=1000,
   #  batch_size=16,
    return_timestamps=True
   )
transcriber.tokenizer.get_decoder_prompt_ids(language='cantonese', task="transcribe")
result = transcriber("source/trimmed_sample.mp3")

now = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
json_object = json.dumps(result, indent=4)
with open('output/'+now+".json", "w") as f:
    f.write(json_object)

# also it will print out the result in the following output block
print(result)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'text': ' 其實都有佢嘅價值啦可能會係同身粉認同有關係啦又或者可能佢會大動到一個地方嘅文化旅游啦佢隕藏住同埋佢對於呢個社會創造緊嘅價值其實都係好重要嘅元素嚟嘅 市單定加一大嘅唐州活發工程已經完成嘅嘞 而市建共亦都話嚟緊會引入共同租住單位嘅共居模式希望底時呢度呢就可以變成一個充滿文化特色同埋活力嘅社區', 'chunks': [{'timestamp': (0.0, 1.4), 'text': ' 其實都有佢嘅價值啦'}, {'timestamp': (1.4, 4.68), 'text': '可能會係同身粉認同有關係啦'}, {'timestamp': (4.68, 8.0), 'text': '又或者可能佢會大動到一個地方嘅文化旅游啦'}, {'timestamp': (8.0, 15.04), 'text': '佢隕藏住同埋佢對於呢個社會創造緊嘅價值其實都係好重要嘅元素嚟嘅'}, {'timestamp': (19.04, 22.64), 'text': ' 市單定加一大嘅唐州活發工程已經完成嘅嘞'}, {'timestamp': (22.64, 26.88), 'text': ' 而市建共亦都話嚟緊會引入共同租住單位嘅共居模式'}, {'timestamp': (0.0, 4.32), 'text': '希望底時呢度呢就可以變成一個充滿文化特色同埋活力嘅社區'}]}


install the following dependencies for plotting and tabulation

```
pip install pandas
```

In [90]:
import pandas as pd
import json
from IPython.display import display


df = pd.json_normalize(result, record_path =['chunks'])
display(df)

# show df in a tablular format



,timestamp,text
0,"(0.0, 1.4)",其實都有佢嘅價值啦
1,"(1.4, 4.68)",可能會係同身粉認同有關係啦
2,"(4.68, 8.0)",又或者可能佢會大動到一個地方嘅文化旅游啦
3,"(8.0, 15.04)",佢隕藏住同埋佢對於呢個社會創造緊嘅價值其實都係好重要嘅元素嚟嘅
4,"(19.04, 22.64)",市單定加一大嘅唐州活發工程已經完成嘅嘞
5,"(22.64, 26.88)",而市建共亦都話嚟緊會引入共同租住單位嘅共居模式
6,"(0.0, 4.32)",希望底時呢度呢就可以變成一個充滿文化特色同埋活力嘅社區
